# We start by importing the relevant libraries

In [924]:
from scapy.all import *
import numpy as np
import graphviz
from graphviz import Source
dotipv4 = graphviz.Digraph(comment='IPv4 Conversations')
dotipv4 #doctest: +ELLIPSIS

## We load the pcap file and split it into Packets and Frames:

In [925]:
all_traffic = rdpcap("./pcap/network-traffic.pcap")
packet_list = []		# to store layer 3 traffic
frame_list = []			# to store layer 2 traffic
for packet in all_traffic:
	if (IP in packet):
		packet_list.append(packet)	
	else:
		frame_list.append(packet)

## We filter the packets, we are interested in only source, destination and size

In [926]:
filtered_packets = []
for packet in packet_list:
	filtered_packets.append([packet['IP'].src + ',' + packet['IP'].dst, len(packet)])
matrix_fp = np.array(filtered_packets)		# matrix of filtered packets

## Identify the conversation (communication between end points)

In [927]:
conversation_list = list(set(matrix_fp[:,0]))
size_list = [0] * len(conversation_list)
matrix_conversations = np.array([conversation_list, size_list])		# size_list gets casted to string =(
matrix_conversations = np.transpose(matrix_conversations)			# change first column to the IP addresses and second column to size

In [928]:
matrix_conversations[0][1]

'0'

### Calculation of the conversation size

In [929]:
for i in range(len(matrix_conversations)):
	conversation_size = 0
	for j in range(len(matrix_fp[:,0])):
		if (matrix_conversations[i,0] == matrix_fp[j,0]):
			conversation_size += int(matrix_fp[j,1])
	matrix_conversations[i,1] = conversation_size

In [930]:
matrix_conversations[:,1]

array(['6516', '2631', '2355', '659', '2030', '487341', '20443', '1454',
       '15182', '1753', '5753', '860'], dtype='<U29')

## We create three vectors for Source, Destination and ConversationSize

In [931]:
src_ips = []
dst_ips = []
con_siz = []
for i in range (len(matrix_conversations)):
	ips = matrix_conversations[i,0].split(",")
	src_ips.append(ips[0])
	dst_ips.append(ips[1])
	con_siz.append(matrix_conversations[i,1])

In [932]:
src_ips_copy = copy.deepcopy(src_ips)
dst_ips_copy = copy.deepcopy(dst_ips)

## We create a dictionary of the conversation with prefix PC<number>

In [933]:
ip_values = src_ips + dst_ips
ip_values = list(set(ip_list))

In [934]:
pc_labels = []
for i in range (len(ip_list)):
	prefix = "PC" + str(i)
	pc_labels.append(prefix)
conversation_dict = dict(zip(ip_values, pc_labels))

## We substitute IP values for their respective key

In [935]:
for item in src_ips:
	index = src_ips.index(item)
	src_ips[index] = conversation_dict.get(item)
for item in dst_ips:
	index = dst_ips.index(item)
	dst_ips[index] = conversation_dict.get(item)

## We create the Digraph of IPv4 Conversations

### We first create the Nodes

In [936]:
for key, value in conversation_dict.items():
	dotipv4.node(key, value)

### We now add the edges

In [937]:
for i in range(len(src_ips_copy)):
	dotipv4.edge(src_ips_copy[i], dst_ips_copy[i], constraint='true')



In [938]:

print(dotipv4.source)

// IPv4 Conversations
digraph {
	"10.139.1.1" [label=PC0]
	"104.23.98.190" [label=PC1]
	"34.117.237.239" [label=PC2]
	"185.199.108.153" [label=PC3]
	"10.138.38.223" [label=PC4]
	"5.135.182.68" [label=PC5]
	"104.23.99.190" [label=PC6]
	"5.135.182.68" -> "10.138.38.223" [constraint=true]
	"10.139.1.1" -> "10.138.38.223" [constraint=true]
	"10.138.38.223" -> "185.199.108.153" [constraint=true]
	"104.23.99.190" -> "10.138.38.223" [constraint=true]
	"10.138.38.223" -> "34.117.237.239" [constraint=true]
	"104.23.98.190" -> "10.138.38.223" [constraint=true]
	"10.138.38.223" -> "104.23.98.190" [constraint=true]
	"10.138.38.223" -> "10.139.1.1" [constraint=true]
	"185.199.108.153" -> "10.138.38.223" [constraint=true]
	"34.117.237.239" -> "10.138.38.223" [constraint=true]
	"10.138.38.223" -> "5.135.182.68" [constraint=true]
	"10.138.38.223" -> "104.23.99.190" [constraint=true]
}



### Rendering the image of the ipv4 conversation:

In [939]:

dotipv4.format = 'svg'
dotipv4.render('test-output/conversation-ipv4.gv')
Source.from_file('test-output/conversation_ipv4.gv.svg')

FileNotFoundError: [Errno 2] No such file or directory: 'test-output/conversation_ipv4.gv.svg'